In [3]:

import os

import pandas as pd
import numpy as np
import h5py

from storage import upload_to_bucket

from schema import Schema, ShoeboxGeometryParameter, BuildingTemplateParameter, WhiteboxSimulation, WindowParameter, SchedulesParameters
from nrel_uitls import ResStockConfiguration


In [4]:
schema = Schema()

In [5]:

basepath = os.path.dirname(os.getcwd())
filepath = os.path.join(basepath, "NREL", "data", "ResStock", "resstock_filtered.csv")
citypath = os.path.join(os.getcwd(), "data", "city_map.json")
resstock_filtered = pd.read_csv(filepath, index_col=0)
resstock_samples = ResStockConfiguration(resstock_filtered, citypath)
sample_df = resstock_samples.get_template_df()
sample_df.head()


Dropped 17 samples while cleaning. Now there are 591 samples.


,bldg_id,archetype,vintage,climate_zone,base_epw,window_type,wwr,heating_setpoint,cooling_setpoint,Infiltration,...,EquipmentPowerDensity,FacadeMass,FacadeRValue,RoofRValue,PartitionRValue,SlabRValue,has_electric_heating,heating_cop,has_cooling,cooling_cop
0,0.0,3.0,1843.0,7.0,0.0,0.0,0.15,20.000000,25.555556,1.000000,...,5.0034,2971.8,0.272143,2.561681,0.0,2.561681,0.0,0.900000,1.0,3.0
1,0.0,2.0,1920.0,9.0,1.0,1.0,0.12,20.000000,18.333333,1.666667,...,6.4022,2971.8,0.272143,5.555693,0.0,5.555693,0.0,0.760000,1.0,3.0
2,0.0,0.0,2010.0,4.0,2.0,3.0,0.09,25.555556,21.111111,0.666667,...,5.7028,280416.0,3.385190,0.272143,0.0,0.272143,1.0,2.930556,1.0,3.0
3,0.0,0.0,2007.0,7.0,3.0,2.0,0.12,22.222222,21.111111,0.466667,...,5.0572,2971.8,3.618392,0.272143,0.0,0.272143,0.0,0.800000,1.0,3.0
4,0.0,0.0,1808.0,9.0,4.0,0.0,0.18,21.111111,23.888889,0.666667,...,2.5555,2971.8,0.272143,0.272143,0.0,0.272143,0.0,0.800000,1.0,3.0


In [6]:
sample_df.columns, schema.parameter_names

(Index(['bldg_id', 'archetype', 'vintage', 'climate_zone', 'base_epw',
        'window_type', 'wwr', 'heating_setpoint', 'cooling_setpoint',
        'Infiltration', 'dhw_flow', 'PeopleDensity', 'LightingPowerDensity',
        'EquipmentPowerDensity', 'FacadeMass', 'FacadeRValue', 'RoofRValue',
        'PartitionRValue', 'SlabRValue', 'has_electric_heating', 'heating_cop',
        'has_cooling', 'cooling_cop'],
       dtype='object'),
 ['batch_id',
  'variation_id',
  'program_type',
  'vintage',
  'climate_zone',
  'base_epw',
  'width',
  'height',
  'facade_2_footprint',
  'perim_2_footprint',
  'roof_2_footprint',
  'footprint_2_ground',
  'wwr',
  'orientation',
  'HeatingSetpoint',
  'CoolingSetpoint',
  'LightingPowerDensity',
  'EquipmentPowerDensity',
  'PeopleDensity',
  'Infiltration',
  'FacadeMass',
  'RoofMass',
  'FacadeRValue',
  'RoofRValue',
  'SlabRValue',
  'WindowSettings',
  'schedules_seed',
  'schedules'])

In [22]:
start_ix = 3
batches_to_gen = 4
batches = []
for batch_id in range(start_ix, start_ix+batches_to_gen):
    base = sample_df.iloc[batch_id]
    storage_batch = schema.generate_empty_storage_batch(1)
    
    # Set Constants
    schema.update_storage_batch(storage_batch,parameter="batch_id",value=batch_id)
    schema.update_storage_batch(storage_batch,parameter="base_epw",value=base.base_epw)
    schema.update_storage_batch(storage_batch,parameter="climate_zone",value=base.climate_zone)
    schema.update_storage_batch(storage_batch,parameter="program_type",value=base.archetype)
    schema.update_storage_batch(storage_batch,parameter="vintage",value=base.vintage)
    
    # Set Defaults
    schema.update_storage_batch(storage_batch, parameter="wwr", value=base.wwr)
    schema.update_storage_batch(storage_batch, parameter="HeatingSetpoint", value=base.heating_setpoint)
    schema.update_storage_batch(storage_batch, parameter="CoolingSetpoint", value=base.cooling_setpoint)
    schema.update_storage_batch(storage_batch, parameter="Infiltration", value=base.Infiltration)
    schema.update_storage_batch(storage_batch, parameter="EquipmentPowerDensity", value=base.EquipmentPowerDensity)
    schema.update_storage_batch(storage_batch, parameter="LightingPowerDensity", value=base.LightingPowerDensity)
    schema.update_storage_batch(storage_batch, parameter="PeopleDensity", value=base.PeopleDensity)
    schema.update_storage_batch(storage_batch, parameter="FacadeMass", value=base.FacadeMass)
    schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=base.FacadeRValue)
    schema.update_storage_batch(storage_batch, parameter="RoofRValue", value=base.RoofRValue)
    schema.update_storage_batch(storage_batch, parameter="SlabRValue", value=base.SlabRValue)

    # TODO: roofmass
    # TODO: Internal mass?
    # TODO: schedule seed

    # Generate Orientations
    storage_batch = storage_batch.repeat(4,axis=0)
    orientations = np.arange(4).reshape(-1,1)
    schema.update_storage_batch(storage_batch, parameter="orientation", value=orientations)

    # geneate geometric variations
    storage_batch = storage_batch.repeat(10,axis=0)
    for j,parameter in enumerate(schema.parameters):
        if isinstance(parameter, ShoeboxGeometryParameter):
            name = parameter.name
            mean = parameter.mean
            std = parameter.std
            shape = parameter.shape_storage
            values = np.random.normal(loc=mean, scale=std, size=(storage_batch.shape[0], *shape))
            schema.update_storage_batch(storage_batch, parameter=name, value=values)
    
    """Generate Schedules"""
    storage_batch = storage_batch.repeat(10, axis=0)


    """Finalize Batch"""
    storage_batch = storage_batch.repeat(5, axis=0)
    n = storage_batch.shape[0]
    print(n)
    # TODO:? Appy specific upgrades? or just use perturbations
    
    """Pertub Geometry"""

    """Perturb Building Parameters"""

    heating_setpoints = schema["HeatingSetpoint"].extract_storage_values_batch(storage_batch)
    shape = schema["HeatingSetpoint"].shape_storage
    heating_setpoints = heating_setpoints + np.random.normal(loc=0, scale=1, size=(n,*shape))
    schema.update_storage_batch(storage_batch, parameter="HeatingSetpoint", value=heating_setpoints)

    shape = schema["CoolingSetpoint"].shape_storage
    cooling_setpoints = schema["CoolingSetpoint"].extract_storage_values_batch(storage_batch)
    cooling_setpoints = cooling_setpoints + np.random.normal(loc=0, scale=1, size=(n,*shape))
    schema.update_storage_batch(storage_batch, parameter="CoolingSetpoint", value=cooling_setpoints)

    shape = schema["CoolingSetpoint"].shape_storage
    infiltration = schema["Infiltration"].extract_storage_values_batch(storage_batch)
    infiltration = infiltration + np.random.normal(loc=-0.2, scale=0.2, size=(n,*shape))
    schema.update_storage_batch(storage_batch, parameter="Infiltration", value=infiltration)

    shape = schema["EquipmentPowerDensity"].shape_storage
    epd = schema["EquipmentPowerDensity"].extract_storage_values_batch(storage_batch)
    epd = epd + np.random.normal(loc=0, scale=2, size=(n, *shape))
    schema.update_storage_batch(storage_batch, parameter="EquipmentPowerDensity", value=epd)

    lpd = schema["LightingPowerDensity"].extract_storage_values_batch(storage_batch)
    lpd = lpd + np.random.normal(loc=0, scale=2, size=(n, *shape))
    schema.update_storage_batch(storage_batch, parameter="LightingPowerDensity", value=lpd)

    ppd = schema["PeopleDensity"].extract_storage_values_batch(storage_batch)
    ppd = ppd + np.random.normal(loc=0, scale=0.03, size=(n, *shape))
    schema.update_storage_batch(storage_batch, parameter="PeopleDensity", value=ppd)

    schema.update_storage_batch(storage_batch, parameter="FacadeMass", value=base.FacadeMass)
    schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=base.FacadeRValue)
    schema.update_storage_batch(storage_batch, parameter="RoofRValue", value=base.RoofRValue)
    schema.update_storage_batch(storage_batch, parameter="SlabRValue", value=base.SlabRValue)

    # Protect Setpoints
    heating_setpoints = schema["HeatingSetpoint"].extract_storage_values_batch(storage_batch)
    cooling_setpoints = schema["CoolingSetpoint"].extract_storage_values_batch(storage_batch)
    heating_setpoints = np.where(heating_setpoints > cooling_setpoints, cooling_setpoints-2, heating_setpoints)
    schema.update_storage_batch(storage_batch, parameter="HeatingSetpoint", value=heating_setpoints)

    # Set Variation ID
    variation_ids = np.arange(n)
    schema.update_storage_batch(storage_batch,parameter="variation_id",value=variation_ids)
    schema.update_storage_batch(storage_batch,parameter="schedules_seed",value=(np.random.random(1))*n*variation_ids)

    batches.append(storage_batch)
    print(storage_batch.shape)

    # # Write to an HDF5 file
    # slug = f"batch_{batch_id:05d}.hdf5"
    # outfile = f"./data/hdf5/{slug}"
    # with h5py.File(outfile,"w") as f:
    #     f.create_dataset(name="storage_vectors", shape=storage_batch.shape, dtype=storage_batch.dtype, data=storage_batch)

    # # upload to cloud bucket for easy backup
    # destination = f"batches/{slug}"
    # upload_to_bucket(destination, outfile)

2000
(2000, 92)
